In [7]:
import os
import random
import spacy

In [31]:
nlp = spacy.load('nl_core_news_sm')

In [3]:
def get_file(num):
    path = './data/clean_articles/'

    names = os.listdir(path)
    file = open(path+names[num],'rt')
    raw_text = file.read()
    file.close()
    return raw_text

# Personen en ministers

We create an annotated text to train the algorithm to analyze legal texts.

In [4]:
Train_people = [
    ('De Minister van de Brusselse Hoofdstedelijke Regering, belast met Financiën, Begroting, Openbaar ambt en de Promotie van Meertaligheid, S. GATZ', { 'entities': [(136, 153, 'PERSON')]}),
    ('De minister is belast met de uitvoering van dit besluit. Brussel, 28 mei 2020. Voor de Regering : De Minister-President, R. VERVOORT De Minister van Economie, A. MARON', {'entities': [(121, 132, 'PERSON'), (159, 167, 'PERSON')]}),
    ('Gezien om gevoegd te worden bij Ons besluit van 22 juni 2020. FILIP Van Koningswege : De Vice-Eerste Minister en Minister van Financiën, A. DE CROO', { 'entities':[(48, 60, 'DATE'),(62, 67, 'PERSON'), (137, 147, 'PERSON')]}),
    ('V. GLATIGNY, Minister van Hoger Onderwijs, Onderwijs voor sociale promotie, Universitaire ziekenhuizen, Hulpverlening aan de Jeugd, Justitiehuizen, Jeugd, Sport en Promotie van Brussel ', {'entities': [(0, 11, 'PERSON')]}),
    ('FILIP Van Koningswege', {'entities': [(0,5, 'PERSON')]}),
    ('De Minister van Middenstand, Zelfstandigen, KMO, Landbouw en Maatschappelijke Integratie D. DUCARME', {'entities': [(89, 99, 'PERSON')]}),
    ('De Minister van Financiën, A. DE CROO', {'entities': [(27, 37, 'PERSON')]}),
    ('De Minister van Werk, Economie en Consumenten, N. MUYLLE', {'entities': [(47,56, 'PERSON')]}),
    ('De minister-president van de Vlaamse Regering, J. JAMBON', {'entities': [(47,56, 'PERSON')]}),
    ('De Vlaamse minister van Economie, Innovatie, Werk, Sociale economie en Landbouw, H. CREVITS', {'entities': [(81, 91, 'PERSON')]}),
    ('De Minister van Werk, Vorming, Gezondheid, Sociale Actie, Gelijke Kansen en Vrouwenrechten, Ch. MORREALE', {'entities': [(92, 104, 'PERSON')]}),
    ('De Minister-President, E. DI RUPO', {'entities': [(23, 33, 'PERSON')]}),
    ('De Minister van Defensie, Ph. GOFFIN', {'entities': [(26, 36, 'PERSON')]}),
    ('Hans D\'HONDT Voorzitter van het Directiecomité', {'entities': [(0, 12, 'PERSON')]}),
    ('De Minister van Economie, Buitenlandse Handel, Onderzoek, Innovatie, Digitale Technologieën, Ruimtelijke Ordening, Landbouw, het "IFAPME", en de Vaardigheidscentra, W. BORSUS', {'entities': [(165, 174, 'PERSON')] }),
    ('De Vice-eersteminister en Minister van Financiën, V. VAN PETEGHEM', {'entities': [(50, 65, 'PERSON')]}),
]

In [5]:
# quick check if annotations are set correctly

for sent in Train_people:
    # print(sent[0][:10])
    for ent in sent[1]['entities']:
        print(sent[0][ent[0]:ent[1]])

S. GATZ
R. VERVOORT
A. MARON
22 juni 2020
FILIP
A. DE CROO
V. GLATIGNY
FILIP
D. DUCARME
A. DE CROO
N. MUYLLE
J. JAMBON
H. CREVITS
Ch. MORREALE
E. DI RUPO
Ph. GOFFIN
Hans D'HONDT
W. BORSUS
V. VAN PETEGHEM


## Training

In [8]:
from spacy.training import Example

In [32]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

In [33]:
for pipe in other_pipes:
    nlp.disable_pipe(pipe)
for i in range(100):
        random.shuffle(Train_people)
        losses = {}
        batches = spacy.util.minibatch(Train_people, size=2)
        for batch in batches:
                for text, annotation in batch:
                        losses = {}
                        # create Example
                        example = Example.from_dict(nlp.make_doc(text), annotation)
                        # Update the model
                        nlp.update([example], losses=losses, drop=0.3)
        print(losses)

{'ner': 0.12171862882431664}
{'ner': 6.23167423408818e-05}
{'ner': 0.945803488746967}
{'ner': 1.2430904164734548e-06}
{'ner': 1.027207034757528e-09}
{'ner': 5.707490897301728e-09}
{'ner': 4.442415159760653e-06}
{'ner': 6.754015743684301e-10}
{'ner': 1.4418883084647198e-14}
{'ner': 1.1353891543057905e-09}
{'ner': 6.960589141791904e-06}
{'ner': 6.119560557018491e-11}
{'ner': 6.536301013904508e-12}
{'ner': 1.842800501655478e-09}
{'ner': 1.0873874034221245e-09}
{'ner': 0.004099525976963241}
{'ner': 1.2463475072180033e-11}
{'ner': 1.2267756097100325e-12}
{'ner': 2.2968688406789127e-08}
{'ner': 1.4088111752219956e-17}
{'ner': 1.2670568036427502e-08}
{'ner': 4.586841254611908e-18}
{'ner': 6.50517474337259e-12}
{'ner': 6.679771690702079e-06}
{'ner': 1.1336367667296384e-13}
{'ner': 3.202645932274119e-12}
{'ner': 1.5194055011677475e-07}
{'ner': 3.4752921297191987e-09}
{'ner': 1.8177787392776475e-15}
{'ner': 6.626632455485992e-15}
{'ner': 1.4788060020118653e-07}
{'ner': 3.802104892444596e-16}
{'n

In [34]:
for pipe in other_pipes:
    nlp.enable_pipe(pipe)

## Testing

One file is loaded to check the annotations

In [101]:
i =  20
doc = nlp(get_file(i))

In [102]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Koning der Belgen PERSON
Aan 
 PERSON
Onze Groet PERSON
1 CARDINAL
74 CARDINAL
2 CARDINAL
1 CARDINAL
6 april 
2020 DATE
3 CARDINAL
2 CARDINAL
9 april 2020 DATE
28 april 2020 DATE
2 CARDINAL
9 april 2020 DATE
3 CARDINAL
9 april 2020 DATE
28 april 2020 DATE
3 CARDINAL
9 april 2020 DATE
3 CARDINAL
13 
 DATE
3 CARDINAL
9 april 2020 DATE
5 CARDINAL
1 CARDINAL
4 CARDINAL
9 april 2020 DATE
mede-eigendom 
 PERSON
28 april 2020 DATE
4 CARDINAL
9 april 2020 DATE
mede-eigendom 
 PERSON
6 CARDINAL
5 CARDINAL
9 april 2020 DATE
27 maart 2020 DATE
2 CARDINAL
31bis CARDINAL
28 november 1969 DATE
27 juni 1969 DATE
28 december 1944 DATE
7 CARDINAL
6 CARDINAL
16 april 2020 DATE
8 CARDINAL
7 CARDINAL
19 april 2020 DATE
9 CARDINAL
19 april 2020 DATE
10 juli 2008 DATE
10 CARDINAL
19 april 2020 DATE
35 CARDINAL
24 juni 2020 DATE
19 april 2020 DATE
11 CARDINAL
11 CARDINAL
22 april 2020 DATE
38 CARDINAL
24 juni 2020 DATE
27 april 2020 DATE
23 maart 2020 DATE
22 december 2016 DATE
13 CARDINAL
14 CARDINAL
27 apr

In [111]:
print(nlp.pipe_names)

['tok2vec', 'morphologizer', 'tagger', 'parser', 'lemmatizer', 'attribute_ruler', 'ner']


#### test on a small doc

In this part I want to have a list of tokens were tokens of one entity are combined

In [76]:
doc_small = nlp('Th. DERMINE De Staatssecretaris voor Digitalisering, belast met \nAdministratieve Vereenvoudiging, Privacy en de Regie der gebouwen, M. MICHEL De Staatssecretaris \nvoor Gendergelijkheid Gelijke Kansen en Diversiteit, S. SCHLITZ De Staatssecretaris \nvoor Asiel en Migratie, belast met de Nationale Loterij')

In [77]:
for ent in doc_small.ents:
    print(ent.text, ent.label_)

Th. DERMINE PERSON
M. MICHEL PERSON
S. SCHLITZ PERSON


In [63]:
[token.lemma_ for token in doc_small if ((token.is_alpha) and (token.pos_ in ['NOUN', 'ADJ' ,'PROPN'])) or (token.ent_id_ != '')]

['DERMINE',
 'staatssecretaris',
 'digitalisering',
 'Administratieve',
 'Vereenvoudiging',
 'Privacy',
 'regie',
 'gebouw',
 'MICHEL',
 'staatssecretaris',
 'Gendergelijkheid',
 'Gelijke',
 'Kansen',
 'Diversiteit',
 'SCHLITZ',
 'staatssecretaris',
 'Asiel',
 'Migratie',
 'nationaal',
 'loterij']

In [82]:
ents = [ent for ent in doc_small.ents]
ents.reverse()
for ent in ents:    
    with doc_small.retokenize() as retokenizer: 
        attrs = {'LEMMA': ent.text, 'is_alpha': True}
        retokenizer.merge(ent, attrs=attrs)

In [83]:
[token.lemma_ for token in doc_small if token.is_alpha and not token.is_stop]

['Th. DERMINE',
 'staatssecretaris',
 'digitalisering',
 'belasten',
 'Administratieve',
 'Vereenvoudiging',
 'Privacy',
 'regie',
 'gebouw',
 'M. MICHEL',
 'staatssecretaris',
 'Gendergelijkheid',
 'Gelijke',
 'Kansen',
 'Diversiteit',
 'S. SCHLITZ',
 'staatssecretaris',
 'Asiel',
 'Migratie',
 'belasten',
 'nationaal',
 'loterij']

### return to the original text

In [121]:
i =  20
doc = nlp(get_file(i))
ents = [ent for ent in doc.ents]
ents.reverse()
for ent in ents:
    if ent.label_ == 'PERSON':    
        with doc.retokenize() as retokenizer: 
            attrs = {'LEMMA': ent.text, 'is_alpha': True}
            retokenizer.merge(ent, attrs=attrs)

In [122]:
tokens = [token.lemma_ for token in doc if token.is_alpha]

In [123]:
tokens[-100:]

['Green',
 'Deal',
 'Z. KHATTABI',
 'de',
 'minister',
 'van',
 'Binnenlandse',
 'Zaken',
 'Institutionele',
 'Hervormingen',
 'en',
 'Democratisch',
 'vernieuwing',
 'A. VERLINDEN',
 'de',
 'minister',
 'van',
 'ontwikkelingssamenwerking',
 'belasten',
 'met',
 'groot',
 'Steden',
 'M. KITIR',
 'de',
 'minister',
 'van',
 'Energie',
 'T. \nVAN DER STRAETEN',
 'de',
 'staatssecretaris',
 'voor',
 'Relance',
 'en',
 'Strategisch',
 'Investering',
 'belasten',
 'met',
 'Wetenschapsbeleid',
 'Th. DERMINE',
 'de',
 'staatssecretaris',
 'voor',
 'digitalisering',
 'belasten',
 'met',
 'Administratieve',
 'Vereenvoudiging',
 'Privacy',
 'en',
 'de',
 'regie',
 'de',
 'gebouw',
 'M. MICHEL',
 'de',
 'staatssecretaris',
 'voor',
 'Gendergelijkheid',
 'Gelijke',
 'Kansen',
 'en',
 'Diversiteit',
 'S. SCHLITZ',
 'de',
 'staatssecretaris',
 'voor',
 'Asiel',
 'en',
 'Migratie',
 'belasten',
 'met',
 'de',
 'nationaal',
 'loterij',
 'S. MAHDI',
 'de',
 'staatssecretaris',
 'voor',
 'Begroting',
 '

In [127]:
for ent in doc.ents:
    if ent.label_ == 'PERSON':
        print(ent.text)

Koning der Belgen
Aan 

Onze Groet
mede-eigendom 

mede-eigendom 

2°
1°
FILIP
A. DE CROO
P.-Y. DERMAGNE
S. WILMES
G. GILKINET
V. VAN PETEGHEM
Fr. VANDENBROUCKE
P. 
DE
V. 
VAN QUICKENBORNE
KMO's en Landbouw
D. CLARINVAL
K. 
LALIEUX
L. DEDONDER
Z. KHATTABI
A. VERLINDEN
M. KITIR
T. 
VAN DER STRAETEN
Th. DERMINE
M. MICHEL
S. SCHLITZ
S. MAHDI
E. DE BLEEKER
V. VAN QUICKENBORNE
Nota
